# Libraries

In [ ]:
import os
import shutil
from random import choice
from time import sleep

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

from IPython.display import display
from fastdownload import download_url
from fastai.vision.widgets import *
from fastai.vision.all import *
from fastcore.all import *

# Data

In [ ]:
path = Path("/kaggle/input/poketwo")

model  = resnet18
epochs = 2

# Training the model

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    batch_tfms=aug_transforms()
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, model, metrics=error_rate)
learn.fine_tune(epochs)

# Test the model

In [ ]:
urls = [
    "https://media.discordapp.net/attachments/720231680564264971/1188445135294435328/pokemon.jpg",
    "https://media.discordapp.net/attachments/720231680564264971/1188444990477713439/pokemon.jpg",
    "https://media.discordapp.net/attachments/720231680564264971/1188443014578843688/pokemon.jpg",
    "https://media.discordapp.net/attachments/926089209268293693/1188420648343703562/pokemon.jpg"
]

for url in urls:
    try:
        download_url(url, "pokemon.jpg", show_progress=False)

        prediction,_,probs = learn.predict(PILImage.create("pokemon.jpg"))

        print(f"Prediction: {prediction} Probability: {max(probs):.4f}")
        display(Image.open("pokemon.jpg").to_thumb(128,128))
    except Exception as e:
        print("Error", e)

# Export the model

In [ ]:
learn.export("model.pkl")

categories = learn.data.label_list.train.x.classes

with open("categories.json", "w") as f:
    json.dump(categories, f)

!ls -la

# Using the model

In [ ]:
trained_learn = load_learner("model.pkl")

In [ ]:
%%timeit

trained_learn.predict(PILImage.create("pokemon.jpg"))